In [4]:
import pandas as pd

In [5]:
prec_hourly = pd.read_csv('data/MODIS/prec_sum.csv')

In [6]:
prec_hourly.head()

,date,hourlyPrecipRate
0,20140301_0000,1.517263
1,20140301_0100,1.041036
2,20140301_0200,0.009600
3,20140301_0300,0.008192
4,20140301_0400,0.004113


In [7]:
dates = prec_hourly['date'].to_list()

Convert to date to datetime objects

In [8]:
new_dates = []
for date in dates:
    new_date = f'{date[0:4]}-{date[4:6]}-{date[6:8]} {date[9:11]}:00:00'
    new_dates.append(new_date)
prec_hourly['date'] = new_dates
prec_hourly['date']= pd.to_datetime(prec_hourly['date'])
prec_hourly = prec_hourly.set_index('date')

In [9]:
prec_hourly

,hourlyPrecipRate
date,
2014-03-01 00:00:00,1.517263
2014-03-01 01:00:00,1.041036
2014-03-01 02:00:00,0.009600
2014-03-01 03:00:00,0.008192
2014-03-01 04:00:00,0.004113
...,...
2017-12-31 19:00:00,0.000000
2017-12-31 20:00:00,0.000000
2017-12-31 21:00:00,0.000000


Resample to daily intervals

In [10]:
prec_daily = prec_hourly.resample('D').sum()

In [11]:
prec_daily

,hourlyPrecipRate
date,
2014-03-01,2.987678
2014-03-02,2.541117
2014-03-03,79.926226
2014-03-04,8.287433
2014-03-05,136.103179
...,...
2017-12-27,0.000000
2017-12-28,29.009759
2017-12-29,385.379790
